In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [21]:
df = pd.read_csv('../data/interim/1.0-ji-initial-data-cleaned')

In [22]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36081 entries, 0 to 36080
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Transaction ID       32466 non-null  object 
 1   Customer ID          32503 non-null  object 
 2   Transaction Date     32473 non-null  object 
 3   Operator Name        36081 non-null  object 
 4   Transaction Type     36081 non-null  object 
 5   Transaction Amount   32507 non-null  float64
 6   Customer Age         32625 non-null  float64
 7   Customer Gender      32501 non-null  object 
 8   Customer Location    32537 non-null  object 
 9   Service Plan         36081 non-null  object 
 10  Data Usage (MB)      32427 non-null  float64
 11  Call Duration (min)  32548 non-null  float64
 12  SMS Sent             32441 non-null  float64
 13  Internet Package     36081 non-null  object 
 14  Transaction Status   36081 non-null  object 
dtypes: float64(5), object(10)
memory usa

,Transaction ID,Customer ID,Transaction Date,Operator Name,Transaction Type,Transaction Amount,Customer Age,Customer Gender,Customer Location,Service Plan,Data Usage (MB),Call Duration (min),SMS Sent,Internet Package,Transaction Status
0,21ca8795-8ad9-47bc-a16b-e4654697ee52,NaN,2023-10-29 02:48:20,9mobile,Bill Payment,4804.10,39.0,other,Kaduna,Postpaid,3440.00,234.30,13.0,Weekly,Failed
1,ea6478bf-6a8c-4d91-bd63-b4464ebc5ae0,3052574b-ae63-4e67-8d86-5f8418ef6f66,2024-04-03 02:22:20,9mobile,Bill Payment,19303.03,NaN,male,Kaduna,Prepaid,NaN,0.90,0.0,Weekly,Completed
2,NaN,15ad55e7-2c9e-4a84-9b9d-9e7d7747a177,2024-02-07 00:22:30,9mobile,Bill Payment,4094.00,NaN,male,Owerri,Prepaid,2250.00,16.38,20.0,Daily,Completed
3,a1165572-b0f4-42e2-9890-eb16ef0b8741,a4adc3b5-2b36-4ba5-b7d9-1a103bfb247a,2008-10-31 21:12:40,Glo,Data Purchase,NaN,36.0,female,Sokoto,Prepaid,3307.97,92.61,58.0,Weekly,Failed
4,07c57156-2d68-433b-92c2-8839db514bc0,NaN,NaN,Glo,Data Purchase,4499.60,49.0,male,Ilorin,Prepaid,3706.69,3.42,59.0,Daily,Pending


**Checking for duplicates in the dataset**

In [28]:
df.duplicated().sum()

np.int64(3272)

In [54]:
# Quick view of duplicated data

df[df.duplicated(keep=False)].sort_values(by='Transaction Date').head()

,Transaction ID,Customer ID,Transaction Date,Operator Name,Transaction Type,Transaction Amount,Customer Age,Customer Gender,Customer Location,Service Plan,Data Usage (MB),Call Duration (min),SMS Sent,Internet Package,Transaction Status
24877,4c31d765-c7e4-4b50-b3ff-11320291d8dc,07220b76-0cf5-4fab-b850-b81835fd8a37,1970-01-08 18:47:21,Airtel,Data Purchase,6877.00,64.0,female,Makurdi,Prepaid,1580.41,197.24,96.0,Monthly,Failed
10172,4c31d765-c7e4-4b50-b3ff-11320291d8dc,07220b76-0cf5-4fab-b850-b81835fd8a37,1970-01-08 18:47:21,Airtel,Data Purchase,6877.00,64.0,female,Makurdi,Prepaid,1580.41,197.24,96.0,Monthly,Failed
20352,3a934b73-e4bc-476a-8e22-36cf06972dbc,47bd1cee-56ad-4a0e-915b-0c52bfa08d88,1970-01-21 12:02:36,9mobile,Data Purchase,NaN,28.0,female,Awka,Prepaid,1169.26,222.92,31.0,Monthly,Failed
34114,3a934b73-e4bc-476a-8e22-36cf06972dbc,47bd1cee-56ad-4a0e-915b-0c52bfa08d88,1970-01-21 12:02:36,9mobile,Data Purchase,NaN,28.0,female,Awka,Prepaid,1169.26,222.92,31.0,Monthly,Failed
26818,399910a7-4d07-4910-8e1c-01a336b7c131,6895bb13-8211-4faf-9e99-c98472754f58,1970-03-15 10:36:24,9mobile,Data Purchase,16996.59,19.0,female,Aba,Postpaid,4242.63,277.34,47.0,Monthly,Failed


In [55]:
# Dropping duplicates
df = df.drop_duplicates()

In [57]:
df.shape

(32809, 15)

**Dropping unncessary columns**

The Transaction ID and Customer ID Columns do not appear to be valuable for future analytics so we can drop them

In [58]:
df[['Transaction ID','Customer ID']].nunique()

Transaction ID    29524
Customer ID       29536
dtype: int64

In [59]:
df = df.drop(columns=['Transaction ID','Customer ID'])